In [1]:
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

In [3]:
#Importing the dataset
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata("MNIST original")
X = mnist["data"]
y = mnist["target"]

X_train = X[:60000]
y_train = y[:60000]
X_test = X[60000:]
y_test = y[60000:]

In [4]:
np.random.seed(42)
#Shuffling the training dataset
rnd_idx = np.random.permutation(60000)
X_train = X_train[rnd_idx]
y_train = y_train[rnd_idx]

In [5]:
#Fitting the linearSVC to the dataset
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [6]:
#Checking the accuracy score
from sklearn.metrics import accuracy_score
y_pred = lin_clf.predict(X_train)
accuracy_score(y_train, y_pred)

0.82633333333333336

In [7]:
#Scaling to remove the outliers
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

In [8]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0)

In [11]:
y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.9224

In [12]:
svm_clf = SVC(decision_function_shape="ovr")
svm_clf.fit(X_train_scaled[:10000], y_train[:10000])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
y_pred = svm_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.94615000000000005

In [18]:
# Now we will use randomized grid search to predict best values of hyperparameters
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_grid = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1,10)}
rnd_search_cv = RandomizedSearchCV(svm_clf, param_grid, n_iter=10, verbose=2)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] C=8.57310093818, gamma=0.0284659124231 ..........................
[CV] ........... C=8.57310093818, gamma=0.0284659124231, total=   1.5s
[CV] C=8.57310093818, gamma=0.0284659124231 ..........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV] ........... C=8.57310093818, gamma=0.0284659124231, total=   1.1s
[CV] C=8.57310093818, gamma=0.0284659124231 ..........................
[CV] ........... C=8.57310093818, gamma=0.0284659124231, total=   1.3s
[CV] C=6.71168895291, gamma=0.0240900860997 ..........................
[CV] ........... C=6.71168895291, gamma=0.0240900860997, total=   1.1s
[CV] C=6.71168895291, gamma=0.0240900860997 ..........................
[CV] ........... C=6.71168895291, gamma=0.0240900860997, total=   1.1s
[CV] C=6.71168895291, gamma=0.0240900860997 ..........................
[CV] ........... C=6.71168895291, gamma=0.0240900860997, total=   1.3s
[CV] C=1.38066683066, gamma=0.00764108097023 .........................
[CV] .......... C=1.38066683066, gamma=0.00764108097023, total=   1.1s
[CV] C=1.38066683066, gamma=0.00764108097023 .........................
[CV] .......... C=1.38066683066, gamma=0.00764108097023, total=   1.1s
[CV] C=1.38066683066, gamma=0.00764108097023 .........................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   46.6s finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'gamma': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2d946b5ba8>, 'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f2d946b5b70>},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring=None, verbose=2)

In [19]:
rnd_search_cv.best_estimator_

SVC(C=3.4414112456607051, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0011460262000260015,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [20]:
rnd_search_cv.best_score_

0.86599999999999999

In [21]:
# Choosing the best estimator
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

SVC(C=3.4414112456607051, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0011460262000260015,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [23]:
#Making the best predictions
y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

0.99436666666666662

In [26]:
#Finally Checking the model on the test set
y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.9718